In [1]:
        !git clone https://github.com/AK18k/lora

Cloning into 'lora'...
remote: Enumerating objects: 1360, done.
remote: Counting objects: 100% (1360/1360), done.
remote: Compressing objects: 100% (966/966), done.
remote: Total 1360 (delta 360), reused 1359 (delta 359), pack-reused 0
Receiving objects: 100% (1360/1360), 23.86 MiB | 19.09 MiB/s, done.
Resolving deltas: 100% (360/360), done.


In [5]:
!pwd

/content


In [12]:
# !cd ./lora/examples/NLG
# !ls '*.sh'
!bash ./lora/examples/NLG/download_pretrained_checkpoints.sh

downloading pretrained model checkpoints...
--2023-06-02 18:24:37--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-pytorch_model.bin
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.225.168, 52.216.26.230, 54.231.202.48, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.225.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548118077 (523M) [application/octet-stream]
Saving to: ‘gpt2-pytorch_model.bin’

gpt2-pytorch_model. 100%[===================>] 522.73M  39.3MB/s    in 12s     

2023-06-02 18:24:50 (43.0 MB/s) - ‘gpt2-pytorch_model.bin’ saved [548118077/548118077]

--2023-06-02 18:24:50--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-medium-pytorch_model.bin
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.230.80, 52.216.105.253, 52.217.226.40, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.230.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520013706 (1.4G) [appli

In [14]:
!bash ./lora/examples/NLG/create_datasets.sh

creating e2e datasets...
train...
Traceback (most recent call last):
  File "/content/lora/examples/NLG/src/gpt2_encode.py", line 31, in <module>
    from progress.bar import Bar as Bar
ModuleNotFoundError: No module named 'progress'
test...
Traceback (most recent call last):
  File "/content/lora/examples/NLG/src/gpt2_encode.py", line 31, in <module>
    from progress.bar import Bar as Bar
ModuleNotFoundError: No module named 'progress'
valid...
Traceback (most recent call last):
  File "/content/lora/examples/NLG/src/gpt2_encode.py", line 31, in <module>
    from progress.bar import Bar as Bar
ModuleNotFoundError: No module named 'progress'
creating webnlg datasets...
train...
cate Airport 1090
cate Astronaut 530
cate Building 972
cate City 243
cate ComicsCharacter 285
cate Food 1424
cate Monument 267
cate SportsTeam 786
cate University 406
cate WrittenWork 937
Traceback (most recent call last):
  File "/content/lora/examples/NLG/src/gpt2_encode.py", line 31, in <module>
    from pro